# **Sudoku Solver**

# Introduction

Many real world problems, can be classified as  constraint Satisfaction Problems. for instance *Scheduling, building design, planning, optimization/satisfaction, Sudoku and Crossword Puzzles and* ...

In a constraint satisfaction problem (CSP) each state is represented in factored way, meaning that there is a set of variables in each state which each of them has a value. The problem is solved when each variable has a value that satisfies all the constraints on that variable.

CSP search algorithms take advantage of the structure of states and use general-purpose heuristics rather than problem-specific heuristics to enable the solution of complex problems.

The main idea is to eliminate large portions of the search space all at once by identifying the variable-value combinations that violate the contraints.

The CSP consist of three components:
* X : set of variables {x1, ..., xn}
* D : set of domains {D1, ..., Dn}
* C : set of constraints {C1, ..., Cn}  Ci=<scope, rel> in which scope is the tuple of variables participating in the constraint and rel is a relation that defines the values that those variable can take on.

There are different algorithms for solving a CSP including:
* **Forward checking**
* **Constraint Propagation (AC-3)**
* **Backtracking**



These problems also can be solved by **Itterative improving algorithms** like local search algorithms (Hillclimbing).
In these algorithms the path to find the goal is irrelevant and the goal state itself is the solution.
In these algorithms, the state space is a set of complete configurations and the task is to find:

1.   Optimal configuration
2.   Configuration satisfying the constraints


 > This project aims to implement different algorithms for solving a Sudoku puzzle including **Backtracking**, **Hillclimbing with random restart** and **Simulated annealing** and comparing the performance of each method by testing them on multiple Sudoku instances.

## Sudoku

Sudoku (meaning single digit in Japanese) originated form 19th century became popular in 80s.


#### Rules




The constraints of Sudoku consists of:
* Each square must have a number between 1-9
* Each number 1-9 can only occure once in a house (  house is the row, column and a block of a cell)


A proper sudoku game has one unique solution and there must be at least 17 clues (Given values) in order to have a unique solution.

The difficulty of the game can be evaluated by:
* Number of Givens
* Average possibilities per empty cell
* Techniques required to solve the puzzle

# Dataset

The dataset used for this project contains 29682 Sudoku puzzles and their solutions which is sourced from kaggle.
It consists of 5 columns:


1.   **id**: Unique id number for each puzzle          
2.   **puzzle**: The puzzle as a string of 81 characters. Periods indicate unknown values
3.   **solution**: The completed Sudoku grid as a string of 81 digits between 1 and 9
4.   **clues**: Number of clues or givens in the puzzle
5.   **difficulty**: Estimated difficulty rating of the puzzle


The level of difficulty varies and most puzzles have between 23 and 26 clues. The minimum number of clues in the dataset is 19, and the maximum is 31. It has been shown that 17 is the minimum number of clues for a valid, uniquely solvable Sudoku puzzle. However, these puzzles are difficult to find, so they are not included in our dataset.

The difficulty rating is computed by an automated solver and it is based on the average search tree depth over 10 attempts. 43% of the puzzles have a difficulty of zero, meaning that it can be solved using a simple scanning technique. The highest difficulty rating is 8.5.

In this project, three levels of difficulty are distinguished by the ratings:

* Easy  [0.0 - 2.5]

* Medium   (2.5 - 4.5)

* Hard   [4.5 - 8.5]

In [ ]:
import random
import numpy as np
import pandas as pd
import copy
import time
import math

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI/sudoku-3m.csv")
df[:-5]

,id,puzzle,solution,clues,difficulty
0,1,1..5.37..6.3..8.9......98...1.......8761.........,1985437266432785915276198439147352688761924352...,27.0,2.2
1,2,...81.....2........1.9..7...7..25.934.2..........,9348172567286534196159427381764258934523981673...,23.0,0.0
2,3,..5...74.3..6...19.....1..5...7...2.9....58..7...,2159837463876542194692713855387169249413258677...,25.0,2.6
3,4,........5.2...9....9..2...373..481.....36....5...,4738169256285397411954278637329481569413652785...,26.0,1.4
4,5,.4.1..............653.....1.8.9..74...24..91.....,9471536821286493576532874913819267455724389164...,25.0,1.1
...,...,...,...,...,...
29674,29675,.....2..1.1.7..92.....8..7.6.2....143....86......,7385924615147639289261843756523798143914286574...,25.0,0.0
29675,29676,....12..6......4.9...3..7....8..5...6..1..3..2...,4739128568215764395693487213187652946951243782...,23.0,2.8
29676,29677,5...........2....3...56..2.1....7.5..8....3..7...,5281734966972485134135697281468372599856123477...,25.0,2.4
29677,29678,.46.2....87.....29..9..1.......7.31...4.8....7...,1467298538736541292598314676829753143142867957...,25.0,0.0


In [ ]:
df.shape[0]


29684

In [ ]:
# Convert . to 0 (for converting the strings to integers) and flat array to 9x9 grid
gridify = lambda puzzle: [ [0 if char == '.' else int(char) for char in puzzle[i * 9:(i + 1) * 9]] for i in range(9)]

In [ ]:
# Get puzzle and solution by index
get_puzzle = lambda index: gridify(df.iloc[index, 1])
get_solution = lambda index: gridify(df.iloc[index,2])

In [ ]:
# Get a random puzzle
def random_puzzle(seed=None):
  random.seed(seed)
  index = random.randrange(29684)
  idx, puzzle, solution, clues, difficulty = df.loc[index]

  return idx-1, gridify(puzzle), gridify(solution), difficulty

random_puzzle()

(7429,
 [[4, 8, 0, 0, 3, 7, 0, 0, 0],
  [0, 2, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 8, 0, 0, 5],
  [0, 3, 0, 0, 0, 4, 1, 0, 0],
  [8, 9, 6, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 5, 9, 0, 3, 7],
  [0, 4, 0, 0, 0, 3, 0, 0, 0],
  [0, 0, 0, 0, 4, 0, 0, 0, 0],
  [0, 6, 0, 0, 0, 0, 8, 9, 0]],
 [[4, 8, 5, 1, 3, 7, 9, 2, 6],
  [3, 2, 9, 4, 6, 5, 7, 1, 8],
  [6, 7, 1, 2, 9, 8, 3, 4, 5],
  [5, 3, 7, 6, 2, 4, 1, 8, 9],
  [8, 9, 6, 3, 7, 1, 4, 5, 2],
  [2, 1, 4, 8, 5, 9, 6, 3, 7],
  [9, 4, 2, 7, 8, 3, 5, 6, 1],
  [1, 5, 8, 9, 4, 6, 2, 7, 3],
  [7, 6, 3, 5, 1, 2, 8, 9, 4]],
 0.0)

In [ ]:
# Print game board
def print_board(grid):
  separator = "+-------+-------+-------+"

  for i, row in enumerate(grid):
    if i % 3 == 0:
      print(separator)

    row_formatted = "| {} {} {} | {} {} {} | {} {} {} |".format(*(str(num) if num != 0 else '.' for num in row))
    print(row_formatted)
  print(separator)

In [ ]:
# Categorize difficulty to 3 level
def categorize_difficulty(difficulty):
  if difficulty <= 2.5:
    return 'easy'
  elif difficulty >= 4.5:
    return 'hard'
  else:
    return 'medium'

# Get indexes of n instance of target difficulty level
def find_puzzles_by_difficulty(df, target_difficulty, n):
  puzzle_indexes = []
  for i, row in df.iterrows():
    difficulty = row['difficulty']
    level = categorize_difficulty(difficulty)
    if level == target_difficulty:
      puzzle_indexes.append(i)
    if len(puzzle_indexes) == n:
      break
  return puzzle_indexes

# **Backtracking**

The term backtracking search is used for a depth-first search that assigns values to one variable at a time, backtracking when no legal values are left. It repeatedly selects an unassigned cell, tries all possible values, and checks for a solution. If an inconsistency is detected, it backtracks to try another value.

The sudoku solving with backtracking algorithm initializes sets to track numbers in rows, columns, and blocks, and identifies empty cells. It checks the validity of placing a number in a cell and recursively tries valid numbers, backtracking as needed. The algorithm measures the time taken and returns the solved board or indicates if no solution is found.

The initialize_sets function initializes and populates sets to track the numbers in each row, column, and 3x3 block of a 9x9 Sudoku grid, and identifies empty cells. It iterates through each cell of the grid, adding numbers to the appropriate sets and recording the coordinates of empty cells. It returns these sets and the list of empty cells for use in solving the Sudoku puzzle.

In [ ]:
def initialize_sets(grid):
  rows = [set() for _ in range(9)]
  cols = [set() for _ in range(9)]
  blocks = [set() for _ in range(9)]
  empty_cells = []

  for i in range(9):
    for j in range(9):
      num = grid[i][j]
      if num != 0:
        rows[i].add(num)
        cols[j].add(num)
        blocks[(i // 3) * 3 + (j // 3)].add(num)
      else:
        empty_cells.append((i, j))

  return rows, cols, blocks, empty_cells

The check_valid function checks if placing a given number in a specified cell of the Sudoku grid is valid. It uses the provided sets for rows, columns, and blocks to determine if the number already exists in the same row, column, or 3x3 block. If the number is found in any of these sets, the function returns False; otherwise, it returns True.

In [ ]:
def check_valid(rows, cols, blocks, row, column, num):
  block_idx = (row // 3) * 3 + (column // 3)
  if num in rows[row] or num in cols[column] or num in blocks[block_idx]:
    return False
  return True

The backtracking function solves the Sudoku grid using a backtracking algorithm. It checks if placing a number in an empty cell is valid, tries each number from 1 to 9, and recursively attempts to solve the puzzle. If a solution is found, it returns True; otherwise, it backtracks and tries another number.

The sudoku_backtracking function initializes the sets for rows, columns, and blocks, and tracks the empty cells. It then calls the backtracking function and measures the time taken to solve the puzzle, returning the solved board and the solving time if successful, or None and the solving time if no solution is found.

In [ ]:
def backtracking(grid, rows, cols, blocks, empty_cells):
  if not empty_cells:
    return True  # Sudoku is solved

  empty_cells.sort(key=lambda cell: len([num for num in range(1, 10) if check_valid(rows, cols, blocks, cell[0], cell[1], num)]))
  i, j = empty_cells.pop(0)

  for num in range(1, 10):  # 1-9
    if check_valid(rows, cols, blocks, i, j, num):
      grid[i][j] = num
      rows[i].add(num)
      cols[j].add(num)
      blocks[(i // 3) * 3 + (j // 3)].add(num)

      if backtracking(grid, rows, cols, blocks, empty_cells):
        return True

      grid[i][j] = 0  # Remove the number if not leading to a solution
      rows[i].remove(num)
      cols[j].remove(num)
      blocks[(i // 3) * 3 + (j // 3)].remove(num)

  empty_cells.insert(0, (i, j))
  return False  # Trigger backtracking

def sudoku_backtracking(grid):
  board = copy.deepcopy(grid)
  rows, cols, blocks, empty_cells = initialize_sets(board)

  start_time = time.time()
  if backtracking(board, rows, cols, blocks, empty_cells):
    end_time = time.time()
    solving_time = end_time - start_time
    return board, solving_time
  else:
    end_time = time.time()
    solving_time = end_time - start_time
    print("No solution is found")
    return None, solving_time

In [ ]:
solution, solving_time = sudoku_backtracking(get_puzzle(42))
if solution == get_solution(42):
  print(f"Solving time: {solving_time:.4f}s")
  print_board(solution)

Solving time: 0.1554s
+-------+-------+-------+
| 1 6 9 | 5 2 4 | 8 7 3 |
| 8 4 5 | 9 7 3 | 1 6 2 |
| 3 2 7 | 8 1 6 | 9 4 5 |
+-------+-------+-------+
| 7 3 4 | 1 8 2 | 5 9 6 |
| 5 8 1 | 3 6 9 | 7 2 4 |
| 2 9 6 | 4 5 7 | 3 1 8 |
+-------+-------+-------+
| 9 5 8 | 6 4 1 | 2 3 7 |
| 4 1 2 | 7 3 5 | 6 8 9 |
| 6 7 3 | 2 9 8 | 4 5 1 |
+-------+-------+-------+


# **Iterative Improving Algorithms**

Iterative improving algorithms such as Hill Climbing and Simulated annealing usualy work with complete states which have all the variables assigned. To apply these algorithms on CSPs, we allow states with unsatisfied constraints and then operators reassign the variable values and try to improve the state iteratively.

## Helper functions


These helper functions are used in both Hillclimbing with Random Restart and Simulated Annealing algorithms.


The initial_solution function fills the empty cells in a 9x9 Sudoku grid with random numbers while ensuring no duplicates in each row. It identifies missing numbers in each row, shuffles them, and assigns them to the empty cells. The function returns the modified grid as an initial candidate solution.

In [ ]:
def initial_solution(grid):
  for i in range(9):
    missing_numbers = [num for num in range(1, 10) if num not in grid[i]]
    random.shuffle(missing_numbers)
    for j in range(9):
      if grid[i][j] == 0:
        grid[i][j] = missing_numbers.pop()
  return grid

The fitness function evaluates the fitness of a 9x9 Sudoku grid by calculating its score based on the uniqueness of numbers in rows, columns, and 3x3 blocks. It sums the count of unique numbers in each row, column, and block. The higher the score, the closer the grid is to a valid Sudoku solution. The function returns this fitness score.

In [ ]:
def fitness(grid):
  score = 0
  for i in range(9):
    score += len(set(grid[i]))  # Row uniqueness
    score += len(set(grid[:, i]))  # Column uniqueness
  for i in range(3):
    for j in range(3):
      block = grid[i*3:(i+1)*3, j*3:(j+1)*3]
      score += len(set(block.flatten()))  # Block uniqueness
  return score

The get_fixed_cells function identifies the fixed cells in a 9x9 Sudoku grid, which are the cells that already contain numbers (i.e., non-zero values). It returns a list of tuples, each representing the coordinates (row, column) of these fixed cells.

In [ ]:
def get_fixed_cells(grid):
  return [(i, j) for i in range(9) for j in range(9) if grid[i][j] != 0]

The get_neighbors function generates neighboring states of a given 9x9 Sudoku grid by swapping non-fixed cells within the same row. It iterates through each non-fixed cell and creates new grids by swapping the values with other non-fixed cells in the same row. It returns a list of these neighboring grids.

In [ ]:
def get_neighbors(grid, fixed):
  neighbors = []
  for i in range(9):
    for j in range(9):
      if (i, j) not in fixed:
        for k in range(j + 1, 9):
          if (i, k) not in fixed:
            neighbor = grid.copy()
            neighbor[i][j], neighbor[i][k] = neighbor[i][k], neighbor[i][j]
            neighbors.append(neighbor)
  return neighbors

## **Hill Climbing with random restart**

The hillclimbing algorithm for solving Sudoku iteratively improves an initial solution (which is achived by randomly filling the input puzzle) by exploring neighboring states and moving to the one with the highest fitness. If no better neighbor is found, it terminates.

It's important to note that the hillclimbing algorithm is not guaranteed to find the optimal solution. It may get stuck in local maxima, where further exploration does not improve the fitness. To mitigate this, additional techniques such as random restarts or incorporating other search algorithms can be employed.

The random restart variant repeatedly applies the hillclimbing algorithm with different initial solutions to find the best possible solution. The algorithm measures and returns the best solution, its fitness, the number of iterations, and the total solving time.

In [ ]:
def hill_climbing(grid, fixed):
  current = np.array(initial_solution(grid))
  current_fitness = fitness(current)
  iterations = 0

  while True:
    neighbors = get_neighbors(current, fixed)
    next_state = max(neighbors, key=fitness)
    next_fitness = fitness(next_state)
    if next_fitness <= current_fitness:
      return current.tolist(), current_fitness, iterations
    current, current_fitness = next_state, next_fitness
    iterations += 1

def sudoku_hillclimbing_random(grid, restarts):
    board = copy.deepcopy(grid)
    fixed = get_fixed_cells(board)
    best_solution = None
    best_fitness = -1
    total_iterations = 0
    start_time = time.time()

    for _ in range(restarts):
      solution, fitness_val, iterations = hill_climbing(board, fixed)
      total_iterations += iterations
      if fitness_val > best_fitness:
        best_solution, best_fitness = solution, fitness_val
      if best_fitness == 243:  # Perfect fitness score
        break

    end_time = time.time()
    solving_time = end_time - start_time
    return best_solution, best_fitness, total_iterations, solving_time

In [ ]:
solution, fitness_val, iterations, solving_time = sudoku_hillclimbing_random(get_puzzle(42),10)
print(f"Fitness: {fitness_val}, Iterations: {iterations}, Solving time: {solving_time:.4f}s")
print_board(solution)

Fitness: 230, Iterations: 170, Solving time: 2.1205s
+-------+-------+-------+
| 9 6 4 | 5 2 8 | 1 7 3 |
| 8 7 5 | 6 1 3 | 9 4 2 |
| 3 2 1 | 9 7 4 | 8 6 5 |
+-------+-------+-------+
| 7 4 6 | 3 8 2 | 5 9 1 |
| 1 8 9 | 5 6 7 | 2 3 4 |
| 2 5 7 | 4 1 9 | 3 8 6 |
+-------+-------+-------+
| 7 9 1 | 2 5 6 | 4 3 8 |
| 8 3 2 | 7 4 5 | 6 1 9 |
| 4 6 3 | 1 9 8 | 7 2 5 |
+-------+-------+-------+


## **Hillclimbing with simulated annealing**

The simulated annealing algorithm for solving Sudoku starts with an initial solution and iteratively explores neighboring states. The algorithm probabilistically accepts worse solutions based on the temperature, which gradually decreases according to the cooling rate. This allows the algorithm to escape local optima. The function returns the best solution found, its fitness, and the number of iterations.

The sudoku_hillclimbing_simulated function initializes the grid and fixed cells, applies the simulated annealing algorithm, and measures the solving time, returning the best solution, its fitness, the number of iterations, and the solving time.

In [ ]:
def simulated_annealing(grid, fixed, initial_temp, cooling_rate, max_iterations):
  current = np.array(initial_solution(grid))
  current_fitness = fitness(current)
  best_solution = current
  best_fitness = current_fitness
  temperature = initial_temp
  iterations = 0

  while temperature > 1 and iterations < max_iterations:
    neighbors = get_neighbors(current, fixed)
    next_state = random.choice(neighbors)
    next_fitness = fitness(next_state)

    if next_fitness > current_fitness:
      current, current_fitness = next_state, next_fitness
    else:
      delta = next_fitness - current_fitness
      probability = math.exp(delta / temperature)
      if random.random() < probability:
        current, current_fitness = next_state, next_fitness

    if current_fitness > best_fitness:
      best_solution, best_fitness = current, current_fitness

    temperature *= cooling_rate
    iterations += 1

  return best_solution.tolist(), best_fitness, iterations

def sudoku_hillclimbing_simulated(grid, initial_temp, cooling_rate, max_iterations):
  board = copy.deepcopy(grid)
  fixed = get_fixed_cells(board)
  start_time = time.time()
  solution, fitness_val, iterations = simulated_annealing(board, fixed, initial_temp, cooling_rate, max_iterations)
  end_time = time.time()
  solving_time = end_time - start_time
  return solution, fitness_val, iterations, solving_time

In [ ]:
solution, fitness_val, iterations, solving_time = sudoku_hillclimbing_simulated(get_puzzle(42),10, 0.99, 10)
print(f"Fitness: {fitness_val}, Iterations: {iterations}, Solving time: {solving_time:.4f}s")
print_board(solution)

Fitness: 202, Iterations: 10, Solving time: 0.0126s
+-------+-------+-------+
| 8 6 1 | 5 2 9 | 4 7 3 |
| 8 7 5 | 9 4 3 | 6 1 2 |
| 3 2 4 | 8 9 7 | 1 6 5 |
+-------+-------+-------+
| 7 3 2 | 4 8 1 | 5 9 6 |
| 7 8 9 | 3 6 1 | 5 2 4 |
| 6 9 5 | 4 8 7 | 3 1 2 |
+-------+-------+-------+
| 7 4 2 | 8 6 5 | 9 3 1 |
| 9 1 8 | 7 3 5 | 6 4 2 |
| 7 5 3 | 4 9 8 | 1 6 2 |
+-------+-------+-------+


# Conclusion


The performance of these algorithms is evaluated based on solving time and fitness score across 600 puzzles of three difficulty levels (easy, medium, and hard). While the backtracking algorithm consistently finds the correct solution, the iterative improving algorithms (Hillclimbing with Random Restart and Simulated Annealing) converge to local optima and do not find the correct solution.

In [ ]:
n = 200
easy_idx = find_puzzles_by_difficulty(df, 'easy', n)
medium_idx = find_puzzles_by_difficulty(df, 'medium', n)
hard_idx = find_puzzles_by_difficulty(df, 'hard', n)

easy_puzzles = [get_puzzle(i) for i in easy_idx]
medium_puzzles = [get_puzzle(i) for i in medium_idx]
hard_puzzles = [get_puzzle(i) for i in hard_idx]

puzzles = {'easy' : easy_puzzles, 'medium' : medium_puzzles, 'hard' : hard_puzzles}

In [ ]:
for difficulty, puzzle_list in puzzles.items():
  results = {'time': []}

  for i in range(n):
    grid = puzzle_list[i]
    solver_solution, solving_time = sudoku_backtracking(grid)

    results['time'].append(solving_time)

  avg_time = sum(results['time']) / n
  print(f"sudoku_backtracking ({difficulty}): Avg Time = {avg_time:.4f}s\n---------------------")

sudoku_backtracking (easy): Average Time = 0.0392s
---------------------
sudoku_backtracking (medium): Average Time = 0.0715s
---------------------
sudoku_backtracking (hard): Average Time = 0.0965s
---------------------


In [ ]:
for difficulty, puzzle_list in puzzles.items():
  results = {'time': [], 'fitness': [], 'iterations': []}

  n=10
  for i in range(n):
    grid = puzzle_list[i]
    solver_solution, best_fitness, iterations, solving_time = sudoku_hillclimbing_random(grid, restarts=10)

    results['time'].append(solving_time)
    results['fitness'].append(best_fitness)
    results['iterations'].append(iterations)

  avg_time = sum(results['time']) / n
  avg_fitness = sum(results['fitness']) / n
  avg_iterations = sum(results['iterations']) / n
  print(f"sudoku_hillclimbing_random ({difficulty}): Avg Time = {avg_time:.4f}s, Avg Fitness = {avg_fitness:.0f}, Avg Iterations = {avg_iterations:.0f}")
  print("-------------------------")

sudoku_hillclimbing_random (easy): Avg Time = 2.2470s, Avg Fitness = 229, Avg Iterations = 166
-------------------------
sudoku_hillclimbing_random (medium): Avg Time = 2.5965s, Avg Fitness = 232, Avg Iterations = 176
-------------------------
sudoku_hillclimbing_random (hard): Avg Time = 2.6014s, Avg Fitness = 230, Avg Iterations = 183
-------------------------


In [ ]:
for difficulty, puzzle_list in puzzles.items():
  results = {'time': [], 'fitness': [], 'iterations': []}

  for i in range(n):
    grid = puzzle_list[i]
    solver_solution, best_fitness, iterations, solving_time = sudoku_hillclimbing_simulated(grid, initial_temp=100, cooling_rate=0.99, max_iterations=1000)

    results['time'].append(solving_time)
    results['fitness'].append(best_fitness)
    results['iterations'].append(iterations)

  avg_time = sum(results['time']) / n
  avg_fitness = sum(results['fitness']) / n
  avg_iterations = sum(results['iterations']) / n
  print(f"sudoku_hillclimbing_simulated ({difficulty}): Avg Time = {avg_time:.4f}s, Avg Fitness = {avg_fitness:.0f} Avg Iterations = {avg_iterations:.0f}")
  print("-------------------------")

sudoku_hillclimbing_simulated (easy): Avg Time = 0.2647s, Avg Fitness = 210 Avg Iterations = 459
-------------------------
sudoku_hillclimbing_simulated (medium): Avg Time = 0.2584s, Avg Fitness = 210 Avg Iterations = 459
-------------------------
sudoku_hillclimbing_simulated (hard): Avg Time = 0.3980s, Avg Fitness = 210 Avg Iterations = 459
-------------------------


# Resources

*   Sudoku rules:

  https://sudoku2.com/sudoku-tips/how-sudoku-difficulty-is-measured/

  https://www.sudokuwiki.org/

  https://www.technologyreview.com/2012/01/06/188520/mathematicians-solve-minimum-sudoku-problem/

* Dataset :
  
  https://www.kaggle.com/datasets/radcliffe/3-million-sudoku-puzzles-with-ratings